## Overview

This script will connect to the EFOQA EMS API URL from a GE CoreLoad machine. 

In order to do this, we will need to set the proxies for the jupyter environment. See [Readme.md in the autorest section](https://github.com/ge-flight-analytics/ems-api-sdk/autorest) of the ems-api-sdk for more information.


In addition to the proxies which tell the browser how to route to the ems api url, we also need to set SSL certificates on the calls themselves. This step is required if the machine is connected to F5 or on the 3.x network. Actually, I'm not sure of all the cases that this is required - i.e. it may be for any call made from a coreload machine.

In order to set the SSL certificate, the only way I could figure is to download the certificate from the GE certificate store. This [yammer thread](https://www.yammer.com/ge.com/#/Threads/show?threadId=1146395629) was helpful in figuring out what needed to be done.


In [ ]:
# Query the user for the credentials for the ems.efoqa.com website.
import getpass
efoqa_user = input('Enter your EFOQA username: ')
efoqa_pass = getpass.getpass(prompt = 'Enter your EFOQA password: ')

In [ ]:
# Import the python autorest wrappers
import sys
sys.path.append("./../../python")
from emsapi import emsapi

# Also import the MS Rest wrappers.
import msrest

# The URL of EFOQA EMS API
emsApiUrl = "https://ems.efoqa.com/api/"

# The relative path to the SSL certificate.
certificatePath = './GE_External_Certificate2.pem'

In [ ]:
# Optional step to ensure that the proxy is set correctly
# This step does not use any of the autorest wrappers or EMS API functionality. 
# If this cell fails to execute, then most likely the proxy is not set up correctly in Jupyter.
import requests
testProxyResponse = requests.get("http://google.com")

# Describe the results of the test
print( "Proxy test is a " + ("tremendous success" if testProxyResponse.ok else "horrible failure") )

In [ ]:
# Authenticates with EMS API using the credentials entered above. 
# This step will call into api and retrieve a bearer token which will be used in subsequent calls.
authorizationUrl = emsApiUrl + "token"
body = "grant_type=password&username=" + efoqa_user +"&password=" + efoqa_pass
authorizationResponse = requests.post(authorizationUrl, body, verify=certificatePath)

# Describe the results of the test
if authorizationResponse.ok:
    print("Successfully authenticated with the EMS API and retrieved a bearer token.")
else:
    print("Unable to authenticate with the EMS API.")   

In [ ]:
# Set up the autorest session with the bearer token.
import json
from msrest import authentication
session = authentication.BasicTokenAuthentication(json.loads(authorizationResponse.text))
myapi = emsapi(session, emsApiUrl)

In [ ]:
# Print the systems the user has access to in order to demonstrate the API.
systems = myapi.ems_system.get_ems_systems(verify=certificatePath)
print("You have access to the following systems:")

import pandas

# Create a list out of the systems list that contains only the information we want.
sysList = []
for system in systems:
    sysList.append( [system.id, system.name, system.description] )

df = pandas.DataFrame(sysList,columns=['id', 'name', 'description'])
df